In [7]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob

In [32]:
# channels = ['miRFP670', 'mRuby3','FITC']
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

# for lens    
#     reader.parser._raw_metadata.image_calibration

# import re
def pimsmeta2OMEXML(reader, project=False):
#     from apeer_ometiff_library import omexmlClass
    import aicsimageio.vendor.omexml as omexmlClass
    import re , os
    
    #Missing TODO:
    #<Image>,  Name = "ImageName"
    #Instrument ID and Detector ID and Objective 
#     frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'wsCameraName']

    # Objective settings with Refractive Index
        #Pixels, 
            #Channel Color = RGB###, EmissionWavelength, Name of Channel.
            #Plane  ExposureTime, Position X, Y, Z (Z is easy as it's in nd2reader metadata)
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):

        if order == 'TZCYX':

            pixel.DimensionOrder = omexmlClass.DO_XYCZT
            counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):

                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
                        #check basically because of triggered acquisition the arrays shouldn't have the size of "channel"
                        pixel.Plane(counter).DeltaT = reader.metadata.PlaneDeltaT(0,counter)
                        pixel.Plane(counter).PositionZ = frames.metadata.PlanePositionZ(0,counter)
                        pixel.Plane(counter).PositionX = frames.metadata.PlanePositionX(0,counter)
                        pixel.Plane(counter).PositionY = frames.metadata.PlanePositionY(0,counter)
#                         pixel.Plane(counter).ExposureTime = 
#                         pixel.Plane(counter).PositionX =
#                         pixel.Plane(counter).PositionY = 
#                         pixel.Plane(counter).
                        counter = counter + 1
                        
    
        return pixel
    
    #make a metadata var
    bfmeta = reader.metadata

    # extra_meta = reader.parser._raw_metadata.image_text_info[b'SLxImageTextInfo'][b'TextInfoItem_5'].decode()
    # extra_meta = re.split(',|;|\r\n',extra_meta)
    # extra_dict = dict()
    # for line in extra_meta: 
        # line = line.strip().strip('- ')
        # keyvalue = str.split(line,':') 
        # if len(keyvalue) > 1: 
            # key = keyvalue[0] 
            # value = keyvalue[1] 
            # extra_dict[key] = value
#     Series = nd2meta['fields_of_view'][-1]+1
    scalex = frames.metadata.PixelsPhysicalSizeX(0)
    scaley = scalex
    
    
    
    if not project:
#         scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
#         scalez = frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep']
        scalez = frames.metadata.PixelsPhysicalSizeZ(0)
    pixeltype = frames.metadata.PixelsType(0)
    # dimorder = 'TZCYX'
    dimorder = frames.metadata.PixelsDimensionOrder(0)
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image_count = 1
#     omexml.image_count = reader.sizes['v']
    #Try to find if PIMS outputs the filename somehow.
    omexml.image(0).Name = os.path.basename(reader.filename)
#     for i in range(frames.sizes['t']):
    p = omexml.image(0).Pixels
    p.SizeX = frames.sizes['x']
#     p.SizeX = 2044
    p.SizeY = frames.sizes['y']
    p.SizeC = frames.sizes['c']
    
    try:
        p.SizeT = frames.sizes['t']
    except: print("Single T")
    
    if project:
        p.SizeZ = 1
    else:
        p.SizeZ = frames.sizes['z']
        
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    if not project:
        p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = frames.sizes['c']
    
    if project:
        p.plane_count = 1 * p.SizeT * p.SizeC #* SizeV
    else:
        p.plane_count = p.SizeZ * p.SizeT * p.SizeC #* SizeV


    #I am using separate files for each visit point
    #, if you want one tiff with all visit points (possibly good for panels) 
    #you will need to update this section
    
    if project:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=1, SizeC=p.SizeC, order=dimorder)
    else:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=p.SizeZ, SizeC=p.SizeC, order=dimorder)
    for c in range(p.SizeC):
        p.Channel(c).Name = str(frames.metadata.ChannelEmissionWavelength(0,c))
        clr = {'miRFP670':  65535 , 'a647': 65535,
               '555 nm' : -16776961,'mRuby3' : -16776961,'mRuby' : -16776961, '630.0': 65535,
               'GFP': 16711935,'FITC': 16711935,'DAPI': 65535,'470 nm': 16711935,}
        p.Channel(c).Color = clr[p.Channel(c).Name]
#         p.Channel(c).EmissionWavelength =
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 2
            
    p.populate_TiffData()
#     omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(p.SizeC))

    return omexml


def Nd2meta2OMEXML(reader, project=False):
#     from apeer_ometiff_library import omexmlClass
    import aicsimageio.vendor.omexml as omexmlClass
    import re
            
            #Missing TODO:
            #<Image>,  Name = "ImageName"
            #Instrument ID and Detector ID and Objective 
            #     frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'wsCameraName']

            # Objective settings with Refractive Index
            #Pixels, 
            #Channel Color = RGB###, EmissionWavelength, Name of Channel.
            #Plane  ExposureTime, Position X, Y, Z (Z is easy as it's in nd2reader metadata)
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):
        if order == 'TZCYX':
            pixel.DimensionOrder = omexmlClass.DO_XYCZT
            counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):
                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
                        #check basically because of triggered acquisition the arrays shouldn't have the size of "channel"
                        pixel.Plane(counter).DeltaT = reader.get_timesteps()[counter//SizeC]
                        pixel.Plane(counter).PositionZ = nd2meta['z_coordinates'][counter//SizeC]
#                         pixel.Plane(counter).ExposureTime = 
#                         pixel.Plane(counter).PositionX =
#                         pixel.Plane(counter).PositionY = 
#                         pixel.Plane(counter).
                        counter = counter + 1
        return pixel
    
    #make a metadata var
    nd2meta = reader.metadata

    extra_meta = reader.parser._raw_metadata.image_text_info[b'SLxImageTextInfo'][b'TextInfoItem_5'].decode()
    extra_meta = re.split(',|;|\r\n',extra_meta)
    extra_dict = dict()
    for line in extra_meta: 
        line = line.strip().strip('- ')
        keyvalue = str.split(line,':') 
        if len(keyvalue) > 1: 
            key = keyvalue[0] 
            value = keyvalue[1] 
            extra_dict[key] = value
#     Series = nd2meta['fields_of_view'][-1]+1
    scalex = nd2meta['pixel_microns']
    scaley = scalex
    
    if not project:
#         scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
#         scalez = frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep']
        scalez = extra_dict['Step'].split()[0]
    pixeltype = 'uint16'
    dimorder = 'TZCYX'
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image_count = 1
#     omexml.image_count = reader.sizes['v']

    omexml.image(0).Name = reader.filename
    p = omexml.image(0).Pixels
    
    p.SizeX = frames.sizes['x']
    p.SizeY = frames.sizes['y']
    p.SizeC = frames.sizes['c']
    p.SizeT = frames.sizes['t']
    
    if project:
        p.SizeZ = 1
    else:
        p.SizeZ = frames.sizes['z']
        
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    if not project:
        p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = frames.sizes['c']
    
    if project:
        p.plane_count = 1 * p.SizeT * p.SizeC #* SizeV
    else:
        p.plane_count = p.SizeZ * p.SizeT * p.SizeC #* SizeV


    #I am using separate files for each visit point
    #, if you want one tiff with all visit points (possibly good for panels) 
    #you will need to update this section
    
    if project:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=1, SizeC=p.SizeC, order=dimorder)
    else:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=p.SizeZ, SizeC=p.SizeC, order=dimorder)
    for c in range(p.SizeC):
        p.Channel(c).Name = nd2meta['channels'][c]
        # try to automate by wavelenght one day
        clr = {'miRFP670':  65535 , 'mirfp670':  65535 , 'a647': 65535,'farRED-EM': 65535, '630.0': 65535,
               'mRuby3' : -16776961,'mRuby' : -16776961,'RED-EM' : -16776961,
               'GFP': 16711935,'FITC': 16711935,'GREEN-EM': 16711935,
               'DAPI': 65535}
        p.Channel(c).Color = clr[p.Channel(c).Name]
    # p.Channel(c).EmissionWavelength =
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 1
            
    p.populate_TiffData()
    # omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(p.SizeC))
    return omexml

def observer(img, i, *args):
    #mgs.append(img.max(axis=0))
    if i % 5 == 0:
        print('Observing iteration = {} (dtype = {}, max = {:.3f})'.format(i, img.dtype, img.max()))   
    #config = tf.ConfigProto(device_count={'GPU': 1})
    #algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=acq.data.ndim, pad_min=[1, 1, 1], session_config=config).initialize()

def init_RL_algo(psfdims,pad_mode='none',pad_min=(0,0,0)):
    return  fd_restoration.RichardsonLucyDeconvolver(n_dims=psfdims
                                                    , pad_mode=pad_mode
    #                                                     , pad_mode='none'
                                                    ,pad_min=pad_min
    #                                                     ,observer_fn=observer
                                                    #,real_domain_fft=True
                                                    #,device='/cpu:0'
                                                   ).initialize()

def deconv(chunk, algo, psf ,iters=20):
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession

    config = ConfigProto()
#     config.gpu_options.allow_growth = True
    # config.gpu_options.per_process_gpu_memory_fraction = 2
    # session = InteractiveSession(config=config)
#     print("Hello")
    return algo.run(fd_data.Acquisition(data=chunk, kernel=psf)
#                                 , session_config = tef.compat.v1.ConfigProto(device_count={'GPU': 0})
                                , session_config=config
#                                 device_count={'GPU': 1}        
#                                 , GPUOptions={'allow_growth' : 4}
#                                         )
                                ,niter=iters
                               ).data



In [41]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end


dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"

filelist = glob.glob(dirname)
filelist.sort()
print(filelist)

frames = pims.bioformats.BioformatsReader(filelist[0],java_memory='1024m')
print(frames.sizes)

metadata = frames.metadata
print(metadata.ChannelEmissionWavelength(3,0))

ydivide = 1
xdivide = 1

if xdivide >1 and ydivide >1:
    depthdivide = (0,64,64)
elif xdivide > 1 and ydivide == 1:
        depthdivide = (0,0,64)
else:
        depthdivide = (0,0,0)
# print(frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep'])


['D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2']
{'x': 1024, 'y': 1024, 'c': 3, 't': 8, 'z': 21}
630.0


C:\Users\Mamede\anaconda3\envs\flowdec\lib\site-packages\pims\bioformats.py:408: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


In [42]:
PSFdir="C:/Users/Mamede Lab/PSF"
# psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
# psfgfp =  tf.imread(PSFdir+'/green.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647
psf647 = np.flip(psf647,
#                  axis=0
                )[8:-8]
# [14:-14]

psfgfp = np.flip(psfgfp,
#                  axis=0
                )[8:-8]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[8:-8]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[8:-8]

print(psf647.shape)
print(psfruby.shape)
# psf647

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


(19, 128, 128)
(19, 128, 128)


In [43]:
algo =init_RL_algo(psfgfp.ndim
                   ,pad_mode='2357',pad_min=(0,1,1)
                  )
filelist

['D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2']

In [44]:
for fname in filelist:
    frames =  pims.bioformats.BioformatsReader(fname,java_memory='1024m')
#     frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels = [frames.metadata.ChannelEmissionWavelength(0,ch) for ch in range(frames.metadata.ChannelCount(0))]
    metadata = frames.metadata
    xml = pimsmeta2OMEXML(frames)
    prjxml = pimsmeta2OMEXML(frames, project=True)

#     psf =[]

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)


    # frames.iter_axes = 'v'
    # for frame in frames:
    #update SERIES HERE
    nseries = metadata.ImageCount()
    for visit in range(nseries):
#     for visit in range(21,31):

        frames.series = visit
        stackfile = fname[:-4]+"_v"+str(visit+1)+'.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1)+'_PRJ.ome.tiff'
        
        with tf.TiffWriter(stackfile, bigtiff=True ,
                           imagej=False) as tif,  tf.TiffWriter(
                        prjfile, bigtiff=True, imagej=False) as tifprj:
            first = True
            for time in range(frames.sizes['t']):

                res = np.zeros(
                    (frames.sizes['z'],frames.sizes['c']
                     ,frames.sizes['y'],frames.sizes['x'])
                    , dtype=np.float32)
                for i in range(frames.sizes['c']):
                    frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                    ch = str(metadata.ChannelEmissionWavelength(0,i))
                    # ['a647', 'DAPI', 'GFP']
                    # ['FITC', 'mRuby3', 'miRFP670']
                    # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                    # ['a647', 'DAPI', 'GFP']
                    if ch == 'DAPI' or ch  == '470 nm':
                        psf = psfdapi
                    elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM':
                        psf = psf647
                    elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM':
                        psf = psfgfp
                    elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
                        psf = psfruby
                    elif ch == 'FRET-gYFP-dsRED':
                        psf = psfruby
                    elif ch == '548.0':
                        psf = psfyfp
                    elif ch == '625.0' or ch == '626.0':
                        psf = psfch
                    else: psf = psfruby

                    arr = da.from_array(frames[time]
    #                                     [:,0:2044,0:2044]
    #                                     [:,:,:]              
                                        , chunks=chunk_size)

#                     print(arr.shape)
                    res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                        dtype='float32', algo=algo,
                                        psf=psf, iters=20
                                                 ).compute(num_workers=1)
#                     res[:,i,:,:] = deconv(frames[time],
# #                                           depth  = depthdivide, boundary='reflect', dtype='float32',
#                                           algo=algo, psf=psf, iters=20
#                              )
#     .compute(num_workers=1)
                    # res[i,:,:,:] = arr.map_blocks(
                    # deconv,dtype='float32').compute(num_workers=1)

                    # If we need to swap Channel and Z in the future
                    #             img5d = np.swapaxes(img5d,0,1)

                    update_progress( (visit+1) * (time+1) / (nseries * frames.sizes['t']) )
                    print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                            visit+1, nseries
                            ,time+1, frames.sizes['t']
                            , i+1, frames.sizes['c']))

                if first:
#                     tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
#                         , description = xml.to_xml()
#                         , photometric='minisblack'
#                         #, datetime= True
#                         , metadata= None
#                         , contiguous=False
#                         )
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                             , compress='LZMA'
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:
#                     tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
# #                         , description = xml.to_xml()
#                         , photometric='minisblack'
#                         , metadata= None
#                         , contiguous=False
#                         )
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                             , compress='LZMA'
#                         , description = xml.to_xml()
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )

        tif.close()
        tifprj.close()

    update_progress(1)

Progress: [####################] 100.0%


In [14]:
# print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

<weakproxy at 0x000001DADC64B6D8 to Device at 0x000001DC7D2C6188>